In [37]:
import multi_asset_var
import numpy as np
import pandas as pd

In [3]:
raw = pd.DataFrame()
raw["CNY"] = pd.read_csv("..\\data\\CNYUSD=X.csv").Close
raw["EUR"] = pd.read_csv("..\\data\\EURUSD=X.csv").Close
returns = raw.pct_change().dropna()

In [4]:
weights = pd.read_csv("..\\data\\wagi.csv")

# Portfele

In [33]:
portfolios = ['real', 'min_risk', 'fixed_return', 'market']

In [34]:
summary = pd.DataFrame(index=['VaR95', 'VaR95 miesięczny', 'VaR99', 'VaR99 miesięczny'])
for portfolio in portfolios:
    var95 = multi_asset_var.var(returns, weights[portfolio], .95)
    var99 = multi_asset_var.var(returns, weights[portfolio], .99)
    var95_monthly = multi_asset_var.var(returns, weights[portfolio], .95, 20)
    var99_monthly = multi_asset_var.var(returns, weights[portfolio], .99, 20)
    summary[portfolio] = [var95, var95_monthly, var99, var99_monthly]

In [36]:
summary.to_csv("..\\data\\var_porfele.csv")
summary

,real,min_risk,fixed_return,market
VaR95,-0.000121,-0.000126,-0.000126,-0.000098
VaR95 miesięczny,-0.002801,-0.002868,-0.002868,-0.002559
VaR99,-0.000111,-0.000117,-0.000117,-0.000082
VaR99 miesięczny,-0.002756,-0.002827,-0.002827,-0.002487


# Proponowane wagi

In [44]:
weights_cny_arr = np.array([0.05, 0.25, 0.4, 0.5, 0.6, 0.75, 0.95])
weights_grid = np.ndarray(shape=(len(weights_cny_arr), 2))
weights_grid[:,0] = weights_cny_arr
weights_grid[:,1] = 1 - weights_cny_arr

In [47]:
summary_grid = pd.DataFrame(index=['VaR95', 'VaR95 miesięczny', 'VaR99', 'VaR99 miesięczny'])
for w in weights_grid:
    var95 = multi_asset_var.var(returns, w, .95)
    var99 = multi_asset_var.var(returns, w, .99)
    var95_monthly = multi_asset_var.var(returns, w, .95, 20)
    var99_monthly = multi_asset_var.var(returns, w, .99, 20)
    summary_grid[w[0]] = [var95, var95_monthly, var99, var99_monthly]

In [50]:
summary_grid.to_csv("..\\data\\var_wybrane_wagi.csv")
summary_grid